In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
files = [x for x in os.listdir('DATADATA1/') if x.endswith('txt')]

In [3]:
data = [pd.read_csv('DATADATA1/' + x, sep='\t') for x in files]

In [4]:
files_withoutExt = [x.split('.')[0] for x in files]

film = [''.join(re.findall('[a-z]', x)) for x in files_withoutExt]

subtitle = [''.join(re.findall('[A-Z]', x)) for x in files_withoutExt]

In [5]:
tmp = []
for counter, df in enumerate(data):
    df = df[['RECORDING_SESSION_LABEL', 'IA_DWELL_TIME', 
             'IA_FIXATION_COUNT', 'IA_DWELL_TIME_%',
             'IA_FIXATION_%', 'IA_LABEL']]
    df['film'] = film[counter]
    df['subtitle'] = subtitle[counter]
    tmp.append(df)

df = pd.concat(tmp, axis=0).reset_index().drop('index', axis=1)

/home/qc/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/qc/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [6]:
pattern = r'[0-9]'
regex = re.compile(pattern)
id = df.iloc[:, 0].apply(lambda x: regex.sub('', x)).tolist()

In [7]:
ind = []
for i in range(df.shape[0]-3):
    if all([id[i] == id[i+1], id[i+1] == id[i+2], id[i+2] == id[i+3]]):
        ind.append(i)
        ind.append(i+1)        

In [8]:
df['RECORDING_SESSION_LABEL'] = id

In [9]:
df = df[~df.index.isin(ind)]

In [10]:
df = df.loc[df.IA_LABEL == 1]

In [26]:
# Testing the missing people from groups

cond_A1 = (df['film'] == 'bj') & (df['subtitle'] == 'IT')
cond_A2 = (df['film'] == 'paris') & (df['subtitle'] == 'CHEN')
cond_A3 = (df['film'] == 'sn') & (df['subtitle'] == 'EN')

cond_B1 = (df['film'] == 'bj') & (df['subtitle'] == 'CHEN')
cond_B2 = (df['film'] == 'paris') & (df['subtitle'] == 'EN')
cond_B3 = (df['film'] == 'sn') & (df['subtitle'] == 'CH')

cond_C1 = (df['film'] == 'bj') & (df['subtitle'] == 'EN')
cond_C2 = (df['film'] == 'paris') & (df['subtitle'] == 'CH')
cond_C3 = (df['film'] == 'sn') & (df['subtitle'] == 'IT')

cond_D1 = (df['film'] == 'bj') & (df['subtitle'] == 'CH')
cond_D2 = (df['film'] == 'paris') & (df['subtitle'] == 'IT')
cond_D3 = (df['film'] == 'sn') & (df['subtitle'] == 'CHEN')

group_A = df.loc[cond_A1 | cond_A2 | cond_A3, 'RECORDING_SESSION_LABEL']
group_B = df.loc[cond_B1 | cond_B2 | cond_B3, 'RECORDING_SESSION_LABEL']
group_C = df.loc[cond_C1 | cond_C2 | cond_C3, 'RECORDING_SESSION_LABEL']
group_D = df.loc[cond_D1 | cond_D2 | cond_D3, 'RECORDING_SESSION_LABEL']

In [27]:
# Have a check on the attendee list
group_A.drop_duplicates()
# hzy, tm are not found in the paper list 

82    dsf
83    hzy
84    mtt
85    syy
86     tm
87    wyh
88    yqy
89     zh
90    zjg
91     zs
92    zyx
Name: RECORDING_SESSION_LABEL, dtype: object

In [28]:
group_B.drop_duplicates()
# lcx should be removed in the paper list
# cx and zpf are not found in the paper list

11    lcx
12     cy
13    cyf
14    lcy
15    lll
16     lm
17    wpp
18     yd
19    zpf
20    zxl
21    zyf
28    wpf
74    hym
Name: RECORDING_SESSION_LABEL, dtype: object

In [23]:
group_C.drop_duplicates()
# yj is not found in the paper list

0       ggy
2        gt
4      hlmy
6       jyj
8       lbj
10      myt
12      syf
14      wyk
16       yc
18       yj
20      zzk
206     zkk
Name: RECORDING_SESSION_LABEL, dtype: object

In [24]:
group_D.drop_duplicates()
# cxt shoulbe be removed in the paper list
# pxh, swd, tt, wyhh, zy, zyj and zjy are not found in the paper list

92      cst
94      dyj
96       lx
98      pxh
100     qwh
102     qxj
104     sll
106     swd
108      tt
110    wyhh
112     zqm
114     zxy
118      zy
120     zyj
222     sdh
228     wyh
230     zjy
Name: RECORDING_SESSION_LABEL, dtype: object

In [16]:
df = df.reset_index().drop('index', axis=1)

In [17]:
updated_index = df[df['RECORDING_SESSION_LABEL'] == 'cx'].index.tolist()

for idx in updated_index:
    df.iat[idx, 0] = 'lcx'

In [18]:
updated_index = df[df['RECORDING_SESSION_LABEL'] == 'wyhh'].index.tolist()

for idx in updated_index:
    df.iat[idx, 0] = 'wyh'

In [29]:
dropped_ids = ['hzy', 'tm', 'zpf', 'hym', 'yj', 'pxh', 'swd', 'tt', 'zy', 'zyj', 'zjy']

In [30]:
filtered_df = df[~df['RECORDING_SESSION_LABEL'].isin(dropped_ids)]
filtered_df['RECORDING_SESSION_LABEL'].drop_duplicates().sort_values()

33      cst
12       cy
13      cyf
60      dsf
34      dyj
0       ggy
1        gt
2      hlmy
3       jyj
4       lbj
11      lcx
14      lcy
15      lll
16       lm
35       lx
62      mtt
5       myt
36      qwh
37      qxj
109     sdh
38      sll
6       syf
63      syy
28      wpf
17      wpp
40      wyh
7       wyk
8        yc
18       yd
66      yqy
67       zh
68      zjg
102     zkk
42      zqm
69       zs
20      zxl
45      zxy
21      zyf
70      zyx
10      zzk
Name: RECORDING_SESSION_LABEL, dtype: object

,RECORDING_SESSION_LABEL,IA_DWELL_TIME,IA_FIXATION_COUNT,IA_DWELL_TIME_%,IA_FIXATION_%,IA_LABEL,film,subtitle


In [45]:


def cronbach_alpha(items):
    items = pandas.DataFrame(items)
    items_count = items.shape[1]
    variance_sum = float(items.var(axis=0, ddof=1).sum())
    total_var = float(items.sum(axis=1).var(ddof=1))
    
    return (items_count / float(items_count - 1) *
            (1 - variance_sum / total_var))
